In [1]:
import pandas as pd
import numpy as np
import torch
import torchtext
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import random
import time
from random import randint
from googletrans import Translator
import re
import openai
import plotly.express as px

In [2]:
# Verificar se o conjunto está ok.

df = pd.read_csv('spam.csv')
df

,Mensagem,Classe
0,"Go until jurong point, crazy.. Available only ...",ham
1,07732584351 - Rodger Burns - MSG = We tried to...,spam
2,No calls..messages..missed calls,ham
3,Congrats! 1 year special cinema pass for 2 is ...,spam
4,Its a part of checking IQ,ham
...,...,...
227,Get your garden ready for summer with a FREE s...,spam
228,Want explicit SEX in 30 secs? Ring 02073162414...,spam
229,I can't believe how attached I am to seeing yo...,ham
230,REMINDER FROM O2: To get 2.50 pounds free call...,spam


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
del(df)

In [ ]:
# Algoritmo para parafrasear o conjunto com o modelo pré-treinado gpt-3.
# Aceita somente o inglês, utilizado a base de spam.

new = []
sentence = "Rewrite a phrase preserving their original meaning: " 

for i in range (0,232):
    var = df[df.index == i]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=sentence+var,
        temperature=0,
        max_tokens=500,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    aux = response.choices[0].text[2:]
    new.append(aux)

In [ ]:
# Algoritmo Random Deletion para o conjunto de spam

new = []

for i in range(0,232):
    var = df[df.index == i]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        if len(spl) <= 5:
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        elif len(spl) <= 35:
            for n in range(0,4):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        else:
            for n in range(0,10):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
    else:
        new.append(var)

In [ ]:
# Algoritmo back translation

translator = Translator()

new = []

for i in range(0,232):
    var = df[df.index == i]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    trans1 = translator.translate(var, dest='pt', src='en').text
    trans2 = translator.translate(trans1, dest='en', src='pt').text
    new.append(trans2)

In [ ]:
# Random Swap

new = []

for i in range(0,232):
    var = df[df.index == i]
    var = var['Mensagem'].to_string()
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        for n in range(0,2):
            rand1 = randint(0,len(spl)-1)
            rand2 = randint(0,len(spl)-1)
            aux1 = spl.pop(rand1)
            spl.insert(rand2, aux1)
        aux2 = ' '.join(spl)
        new.append(aux2)
    else:
        new.append(var)

In [ ]:
novo = pd.DataFrame(new)
novo['Classe'] = df['Classe']
novo.rename(columns={0: 'Mensagem'}, inplace = True)
novo

In [ ]:
novo = novo.drop(232)


In [ ]:
novo

In [ ]:
fim = df.append(novo, ignore_index=True)

In [ ]:
del(df)

In [ ]:
del(novo)

In [ ]:
fim = fim.sample(frac = 1)

In [ ]:
fim = fim.reset_index(drop=True)
fim

In [ ]:
fim.to_csv('spamBT.csv', index=None)

In [ ]:
#out = df.sample(n = 1250)
#out

In [ ]:
#df = df.drop(out.index)

In [ ]:
#df = df.drop(out.index)
#df

In [ ]:
#df = df.reset_index(drop=True)
#df

In [ ]:
#df.to_csv('spam3.csv', index=None)

In [ ]:
#df = df[['Mensagem', 'Classe']]

In [ ]:
#df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [3]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

LEARNING_RATE = 0.005
BATCH_SIZE = 100
NUM_EPOCHS = 85
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 100
HIDDEN_DIM = 256
NUM_CLASSES = 2

In [4]:
### Defining the feature processing

TEXT = torchtext.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.data.LabelField(dtype=torch.long)

In [5]:
fields = [('Mensagem', TEXT), ('Classe', LABEL)]

dataset = torchtext.data.TabularDataset(
    path='spam.csv', format='csv',
    skip_header=True, fields=fields)

In [7]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 186
Num Test: 46


In [8]:
print(vars(train_data.examples[0]))

{'Mensagem': ['Where', 'u', 'been', 'hiding', 'stranger', '?'], 'Classe': 'ham'}


In [9]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 1744
Number of classes: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

In [ ]:
print(TEXT.vocab.itos[:10]) # itos = integer-to-string

In [10]:
print(LABEL.vocab.stoi)

defaultdict(None, {'spam': 0, 'ham': 1})


In [11]:
LABEL.vocab.freqs

Counter({'ham': 87, 'spam': 99})

In [ ]:
train_loader ,test_loader = \
        torchtext.data.BucketIterator.splits(
        (train_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=True,
         sort_key=lambda x: len(x.Mensagem),
         device=DEVICE
    )

In [ ]:
for i in enumerate(train_loader):
    print(i)

In [ ]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.Mensagem.size()}')
    print(f'Target vector size: {batch.Classe.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.Mensagem.size()}')
    print(f'Target vector size: {batch.Classe.size()}')
    break

In [ ]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

In [ ]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
    return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()

treinamento = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.Mensagem.to(DEVICE)
        labels = batch_data.Classe.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        resultado = compute_accuracy(model, train_loader, DEVICE)
        print(f'training accuracy: 'f'{compute_accuracy(model, train_loader, DEVICE):.2f}%')
        treinamento.append(resultado.item())
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

In [ ]:
epocas = [range(1, 86)]

In [ ]:
fig = px.line(x = epocas, y = treinamento, title='Treinamento SMS Spam + BT')
fig.update_yaxes(title='Acurácia')
fig.update_xaxes(title='Épocas')
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
import spacy


nlp = spacy.blank("en")

def predict(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    prediction = torch.max(prediction, 1)
    return prediction

In [ ]:
predict(model, "do you want a gift, click here")